# 🚀 PPO Ultimate Training - Gold Trading AI

**Notebook optimizado para entrenamientos largos sin desconexiones**

---

## ✅ Características:
- **Setup rápido** (~30 segundos via Git Clone)
- Anti-desconexión mejorado
- Entrenamiento en disco LOCAL
- Copia automática a Drive al finalizar

---

## ⚙️ Setup:
1. **GPU**: Runtime → Change runtime type → **GPU (A100 recomendado)**
2. **Ejecutar celdas en orden**
3. **IMPORTANTE**: Ejecuta el código anti-desconexión en F12

## 0️⃣ ANTI-DESCONEXIÓN (MUY IMPORTANTE)

### Método 1: Código en Consola del Navegador (MÁS EFECTIVO)

1. Presiona **F12** para abrir DevTools
2. Ve a la pestaña **Console**
3. Pega este código y presiona Enter:

```javascript
function ClickConnect() {
    console.log('🔄 Keeping alive... ' + new Date().toLocaleTimeString());
    var connectBtn = document.querySelector('#top-toolbar > colab-connect-button');
    if (connectBtn && connectBtn.shadowRoot) {
        var innerBtn = connectBtn.shadowRoot.querySelector('#connect');
        if (innerBtn) innerBtn.click();
    }
    document.dispatchEvent(new MouseEvent('mousemove'));
    document.dispatchEvent(new KeyboardEvent('keypress'));
}
setInterval(ClickConnect, 60000);
console.log('✅ Anti-desconexión activado!');
```

### Método 2: Ejecuta la siguiente celda (backup)

In [ ]:
import IPython
from IPython.display import display, Javascript

js_code = '''
function ClickConnect() {
    console.log("🔄 Keep-alive: " + new Date().toLocaleTimeString());
    var selectors = ["colab-connect-button", "#top-toolbar > colab-connect-button"];
    for (var i = 0; i < selectors.length; i++) {
        var btn = document.querySelector(selectors[i]);
        if (btn && btn.shadowRoot) {
            var innerBtn = btn.shadowRoot.querySelector("#connect");
            if (innerBtn) { innerBtn.click(); break; }
        }
    }
    document.dispatchEvent(new MouseEvent("mousemove", {bubbles: true}));
}
setInterval(ClickConnect, 45000);
ClickConnect();
console.log("✅ Anti-desconexión Python activado!");
'''
display(Javascript(js_code))
print("✅ Anti-desconexión activado!")
print("⚠️  También ejecuta el código en F12 → Console para máxima efectividad")

## 1️⃣ Montar Google Drive (para datos y guardar resultados)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Clonar Proyecto desde GitHub (RÁPIDO ~30 segundos)

**¿Por qué Git Clone?**
- Solo descarga código (~1MB) - NO los datos pesados
- Los datos se leen directamente desde Drive (symlink)
- Siempre tienes la última versión del código

In [ ]:
import os
import shutil
import sys

# IMPORTANTE: Ir a directorio base primero (evita errores si se re-ejecuta)
os.chdir('/content')

LOCAL_PATH = '/content/XAUX'
DRIVE_DATA = '/content/drive/MyDrive/XAUX/data'
GITHUB_REPO = 'https://github.com/alejoamorocho/xuax.git'

# Limpiar si existe
if os.path.exists(LOCAL_PATH):
    shutil.rmtree(LOCAL_PATH)

# Clonar desde GitHub (solo código, ~30 segundos)
print("📥 Clonando desde GitHub...")
!git clone --depth 1 {GITHUB_REPO} {LOCAL_PATH}

# Verificar que existe la carpeta de datos en Drive
if not os.path.exists(DRIVE_DATA):
    print(f"\n❌ ERROR: No existe la carpeta de datos: {DRIVE_DATA}")
    print("\n📁 Asegúrate de tener estos archivos en tu Drive:")
    print("   /MyDrive/XAUX/data/XAUUSD_M5.csv")
    print("   /MyDrive/XAUX/data/economic_events.json")
    print("   /MyDrive/XAUX/data/*.csv (datos macro)")
else:
    # Eliminar carpeta data vacía y crear symlink a Drive
    local_data = os.path.join(LOCAL_PATH, 'data')
    if os.path.exists(local_data):
        shutil.rmtree(local_data)
    os.symlink(DRIVE_DATA, local_data)
    print(f"\n🔗 Datos vinculados desde Drive")

# Agregar al Python path
if LOCAL_PATH not in sys.path:
    sys.path.insert(0, LOCAL_PATH)

os.chdir(LOCAL_PATH)
print(f"\n✅ Proyecto listo!")
print(f"📂 Directorio: {os.getcwd()}")

## 3️⃣ Instalar Dependencias

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q stable-baselines3[extra]
!pip install -q pandas numpy tqdm matplotlib seaborn gymnasium yfinance
print("\n✅ Dependencias instaladas!")

## 4️⃣ Verificar GPU

In [ ]:
import torch

print(f"🖥️  PyTorch: {torch.__version__}")
print(f"🎮 CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {gpu_name} ({gpu_mem:.1f} GB)")
    device = 'cuda'
else:
    print("⚠️  GPU no disponible, usando CPU")
    device = 'cpu'

print(f"\n✅ Device: {device}")

## 5️⃣ Configuración

In [ ]:
TIMESTEPS = 3_000_000  # 3M recomendado
WINDOW = 128  # 10.6 horas de contexto
DEVICE = device

print("⚙️  Configuración PPO:")
print("="*50)
print(f"📊 Timesteps: {TIMESTEPS:,}")
print(f"🪟 Window: {WINDOW} barras ({WINDOW * 5 / 60:.1f} horas)")
print(f"🖥️  Device: {DEVICE}")
print("="*50)

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    h = TIMESTEPS/1e6 * (3 if 'A100' in gpu else 5 if 'V100' in gpu or 'T4' in gpu else 4)
    print(f"\n⏱️  ~{h:.1f} horas en {gpu}")

## 5.5 (Opcional) Diagnostico Rapido

Ejecuta esto ANTES del entrenamiento completo para verificar:
- Si los features son predictivos
- Si el modelo puede aprender de los winners
- Por que el win rate puede ser bajo

**Solo toma ~10-15 minutos**

In [ ]:
# OPCIONAL: Ejecutar diagnostico antes del entrenamiento completo
# Esto analiza si los features son predictivos y si el modelo puede aprender

print("🔍 Ejecutando diagnóstico de aprendizaje...")
print("   Esto toma ~10-15 minutos\n")

!cd /content/XAUX && PYTHONPATH=/content/XAUX python train/diagnose_learning.py --steps 200000 --device {device}

print("\n" + "="*60)
print("📋 INTERPRETACION:")
print("="*60)
print("""
- Si hay POCOS features con effect size > 0.5:
  Los features no distinguen winners de losers -> modelo no puede aprender

- Si win rate NO mejora entre Q1 y Q4:
  El modelo no esta aprendiendo -> revisar reward function

- Si losers duran MAS que winners:
  Modelo esta cortando winners y dejando correr losers -> invertir logica
""")

## 6️⃣ Entrenar PPO

In [ ]:
!cd /content/XAUX && PYTHONPATH=/content/XAUX python train/train_ppo_ultimate.py --timesteps {TIMESTEPS} --window {WINDOW} --device {DEVICE}

# ===== AUTO-SAVE A DRIVE AL TERMINAR =====
import shutil, os, glob

LOCAL = '/content/XAUX/train/ppo_ultimate'
DRIVE = '/content/drive/MyDrive/XAUX/train/ppo_ultimate'

if os.path.exists(LOCAL):
    os.makedirs(DRIVE, exist_ok=True)
    print("\n" + "="*50)
    print("📤 AUTO-GUARDANDO EN DRIVE...")
    print("="*50)
    
    for f in ['ppo_ultimate_final.zip', 'ppo_ultimate_metrics.json', 'vec_normalize.pkl']:
        src = f'{LOCAL}/{f}'
        if os.path.exists(src):
            shutil.copy2(src, f'{DRIVE}/{f}')
            print(f"   ✅ {f}")
    
    # Último checkpoint
    cps = sorted(glob.glob(f'{LOCAL}/*.zip'))
    if cps:
        shutil.copy2(cps[-1], f'{DRIVE}/{os.path.basename(cps[-1])}')
        print(f"   ✅ {os.path.basename(cps[-1])}")
    
    print("\n🎉 MODELO GUARDADO EN DRIVE!")
    print(f"📂 {DRIVE}")
else:
    print("⚠️  No se encontró carpeta de entrenamiento")

## 7️⃣ Copiar Resultados a Drive

In [ ]:
import shutil, os, glob

LOCAL = '/content/XAUX/train/ppo_ultimate'
DRIVE = '/content/drive/MyDrive/XAUX/train/ppo_ultimate'
os.makedirs(DRIVE, exist_ok=True)

print("📤 Copiando a Drive...")
for f in ['ppo_ultimate_final.zip', 'ppo_ultimate_metrics.json', 'vec_normalize.pkl']:
    src = f'{LOCAL}/{f}'
    if os.path.exists(src):
        shutil.copy2(src, f'{DRIVE}/{f}')
        print(f"   ✅ {f}")

# Último checkpoint
cps = sorted(glob.glob(f'{LOCAL}/*.zip'))
if cps:
    shutil.copy2(cps[-1], f'{DRIVE}/{os.path.basename(cps[-1])}')
    print(f"   ✅ {os.path.basename(cps[-1])}")

print("\n🎉 Guardado en Drive!")

## 8️⃣ Ver Resultados

In [ ]:
import json, os

for p in ['/content/XAUX/train/ppo_ultimate/ppo_ultimate_metrics.json',
          '/content/drive/MyDrive/XAUX/train/ppo_ultimate/ppo_ultimate_metrics.json']:
    if os.path.exists(p):
        with open(p) as f: m = json.load(f)
        print("="*60)
        print("📊 RESULTADOS PPO")
        print("="*60)
        print(f"\nVALIDATION: {m['validation']['total_return']:.1f}% | Sharpe: {m['validation']['sharpe_ratio']:.2f}")
        print(f"TEST:       {m['test']['total_return']:.1f}% | Sharpe: {m['test']['sharpe_ratio']:.2f}")
        break
else:
    print("❌ Métricas no encontradas")

## 9️⃣ Resumir (si se interrumpe)

In [ ]:
import glob, os
for d in ['/content/XAUX/train/ppo_ultimate', '/content/drive/MyDrive/XAUX/train/ppo_ultimate']:
    if os.path.exists(d):
        cps = [c for c in sorted(glob.glob(f'{d}/*.zip')) if 'final' not in c]
        if cps:
            print(f"📁 Último checkpoint: {os.path.basename(cps[-1])}")
            print(f"\n🔄 Para resumir:")
            print(f"!cd /content/XAUX && PYTHONPATH=/content/XAUX python train/train_ppo_ultimate.py --timesteps {TIMESTEPS} --window {WINDOW} --device {device} --resume {cps[-1]}")
            break

---
## 📝 Tips
1. Ejecuta JavaScript en F12 → Console
2. Mantén la pestaña visible
3. Revisa cada 1-2h por CAPTCHAs

🎉 **¡Listo!**